# Add new potentials

This Notebook is designed to facilitate the easy integration of new potentials into the calculation framework. The basic outline of steps is:

1. Generate a record for the potential and test that it works properly.

2. Save the record and any associated parameter files to the library directory.

3. Upload the new potential(s) to pre-defined iprPy databases.

__Library imports__

In [1]:
# Standard Python libraries
import os

# https://github.com/usnistgov/DataModelDict
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.lammps as lmp

# https://github.com/usnistgov/iprPy
import iprPy

## 1. Generate new potential record

The iprPy.tools.generate_potential_record() function will build the content for a potential record based on a few necessary inputs.  Examples are shown here for a few different pair_styles.  Not all pair_styles supported yet by the generation function.

Any "accelerated" pair_style should use the associated non-accelerated format, i.e. ignore '/gpu', '/intel', '/kk', '/omp', '/opt'.

Common Parameters

- **id** (*str*) Unique id to refer to the potential implementation (i.e. version).

- **key** (*str, optional*) UUID key for the potential implementation.  If not given, a new UUID4 will be generated.

- **pot_id** (*str, optional*) Unique id to refer to the potential concept (i.e. mathematical model).  If not given, will be based on id.

- **pot_key** (*str, optional*) UUID key for the potential concept.  If not given, a new UUID4 will be generated.

- **units** (*str, optional*) LAMMPS units option.  Default value is 'metal'.

- **atom_style** (*str, optional*) LAMMPS atom_style option.  Default value is 'atomic'.

- **pair_style** (*str*) LAMMPS pair_style option.

- **pair_style_terms** (*list*) List of any other terms that should be included in the LAMMPS pair_style command line.

- **elements** (*str or list, optional*) List of element tags associated with each atomic model in the potential.

- **masses** (*float or list, optional*) List of elemental masses associated with each atomic model in the potential.  Should be given if the potential itself specifies the mass to use.  Note that missing masses will be assigned standard elemental values when the record is loaded by atomman.lammps.Potential.

- **symbols** (*str or list, optional*) List of identifying symbols associated with each atomic model in the potential.  Only required when the element tags differ from the model identifiers. 

### 1.1. Potentials with a single parameter file

For pair_styles 'adp', 'agni', 'airebo', 'airebo/morse', 'rebo', 'bop', 'comb', 'comb3', 'eam/alloy', 'eam/cd',  'eam/fs', 'edip', 'edip/multi', 'extep', 'gw', 'gw/zbl', 'lcbop', 'meam/spline', 'meam/sw/spline', 'nb3b/harmonic', 'polymorphic', 'reax', 'reax/c', 'smtbq', 'sw', 'table', 'table/rx', 'tersoff', 'tersoff/table', 'tersoff/mod', 'tersoff/mod/c', 'tersoff/zbl', 'vashishta' and 'vashishta/table'

Format-Specific Parameters

- **paramfile** (*str*) The filename for the potential's parameter file.

In [2]:
kwargs = {}
kwargs['id'] = '1999--Mishin-Y--Al--LAMMPS--testing1'

kwargs['elements'] = ['Al']
kwargs['masses'] = [26.982]

kwargs['pair_style'] = 'eam/alloy'
kwargs['paramfile'] = 'Al99.eam.alloy'

content = iprPy.tools.generate_potential_record(**kwargs)

### 1.2 Potentials with library (and param) file(s)

For pair_styles 'meam', 'meam/c', 'snap'

Format-Specific Parameters

- **libfile** (*str*) The filename for the potential's library file.

- **paramfile** (*str, optional*) The filename for the potential's parameter file, if it has one.

In [ ]:
kwargs = {}
kwargs['id'] = 'Sn-MEAM-test1'

kwargs['elements'] = 'Sn'

kwargs['pair_style'] = 'meam'
kwargs['libfile'] = 'library.meam'
kwargs['paramfile'] = 'Sn.meam'

content = iprPy.tools.generate_potential_record(**kwargs)

### 1.3 Original EAM potentials

Only for pair_style 'eam'

Format-Specific Parameters

- **paramfiles** (*list, optional*) The filenames for the potential's parameter files correspodning to each included element model.

In [ ]:
kwargs = {}
kwargs['id'] = 'Al-EAM-test1'

kwargs['elements'] = ['Al']
kwargs['masses'] = [26.982]

kwargs['pair_style'] = 'eam'
kwargs['paramfiles'] = ['Al_1.eam']


content = iprPy.tools.generate_potential_record(**kwargs)

### 1.4 Look at content

In [3]:
print(content.json(indent=4))

{
    "potential-LAMMPS": {
        "key": "068f3dca-0e9f-4774-a420-63bcd9f53a04", 
        "id": "1999--Mishin-Y--Al--LAMMPS--testing1", 
        "potential": {
            "key": "0886d7a5-80e5-4df3-814a-c3347388f954", 
            "id": "1999--Mishin-Y--Al"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": {
            "element": "Al", 
            "mass": 26.982
        }, 
        "pair_style": {
            "type": "eam/alloy"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Al99.eam.alloy"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}


### 1.5 Test reading the content

In [4]:
potential = lmp.Potential(content)
print(potential.id)
print(potential.potid)
print(potential.pair_info())

1999--Mishin-Y--Al--LAMMPS--testing1
1999--Mishin-Y--Al
mass 1 26.982000

pair_style eam/alloy
pair_coeff * * Al99.eam.alloy Al



## 2. Save potential record

The iprPy.tools.save_potential_record() function saves a potential record to the library directory so that iprPy databases can upload it later.

Parameters
    
- **content** (*str or DataModelDict.DataModelDict*) The record content to save to the library directory.  Can be xml/json content, path to an xml/json file, or a DataModelDict.

- **files** (*str or list, optional*) The directory path(s) to the parameter file(s) that the potential uses.

- **lib_directory** (*str, optional*) The directory path for the library.  If not given, then it will use the iprPy/library directory.

- **record_style** (*str, optional*) The record_style to save the record as.  Default value is 'potential_users_LAMMPS', which should be used for user-defined potentials.

- **replace** (*bool, optional*) If False (Default), will raise an error if a record with the same title already exists in the library.  If True, any matching records will be overwritten.

In [5]:
files = ['Al99.eam.alloy']
iprPy.tools.save_potential_record(content, files=files)

ValueError: Record 1999--Mishin-Y--Al--LAMMPS--testing1 already exists

## 3. Copy new potentials to a database

Once saved into the library directory, the new potential(s) can be uploaded to a database using the build_refs() option. 

In [6]:
database = iprPy.load_database('potential_testing')

In [7]:
#database.destroy_records('potential_users_LAMMPS')
database.build_refs()

In [7]:
database.check_records('potential_users_LAMMPS')

In database style local at C:\Users\lmh1\Documents\calculations\ipr\potential_testing :
- 1 of style potential_users_LAMMPS
